### Fine tuning the weights
In this section, we'll unfreeze the pre-trained weights of the network and allow them to change.  
Be careful - when fine-tuning a network, there is a risk that our attempt to allow the network to adapt to the new domain will lead to a "catastrophic forgetting" of what it had previously learnt.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import models

import numpy as np

BATCH_SIZE = 8
NUM_WORKERS = 2

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=NUM_WORKERS)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=NUM_WORKERS)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def evaluate_net(net):

    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images.to(device))
            _, predicted = torch.max(outputs.data, 1)
            c = (predicted == labels.to(device)).squeeze()

            for i in range(BATCH_SIZE):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    [print(f'Accuracy of {c}: \t {100 * class_correct[j] / class_total[j]}%') 
     for j, c in enumerate(classes)];

    print('\nAccuracy of the network on the 10000 test images:',  
          f'{100 * np.sum(class_correct) / np.sum(class_total)}%')



Files already downloaded and verified
Files already downloaded and verified


In [2]:

device = "cuda" if torch.cuda.is_available() else "cpu"

model = models.vgg16(pretrained=True)

for param in model.parameters():
    param.requires_grad = True
    
torch.manual_seed(42)
if device == 'cuda':
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Add on classifier
model.classifier[6] = nn.Sequential(
                      nn.Linear(4096, 256), 
                      nn.ReLU(), 
                      nn.Linear(256, 10),                   
                      nn.LogSoftmax(dim=1))

for param in model.classifier[6].parameters():
    param.requires_grad = True

model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.00005)
criterion = nn.CrossEntropyLoss()


for epoch in range(4):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data_train in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data_train[0].to(device), data_train[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print(f'epoch: {epoch + 1}\t iter: {i + 1}\t loss:{running_loss / 1000:.5f}')
            running_loss = 0.0

print('Finished Training')

evaluate_net(model)

epoch: 1	 iter: 1000	 loss:1.09049
epoch: 1	 iter: 2000	 loss:0.72312
epoch: 1	 iter: 3000	 loss:0.65535
epoch: 1	 iter: 4000	 loss:0.60588
epoch: 1	 iter: 5000	 loss:0.57770
epoch: 1	 iter: 6000	 loss:0.53489
epoch: 2	 iter: 1000	 loss:0.39499
epoch: 2	 iter: 2000	 loss:0.38161
epoch: 2	 iter: 3000	 loss:0.37437
epoch: 2	 iter: 4000	 loss:0.38248
epoch: 2	 iter: 5000	 loss:0.37894
epoch: 2	 iter: 6000	 loss:0.36569
epoch: 3	 iter: 1000	 loss:0.24428
epoch: 3	 iter: 2000	 loss:0.24047
epoch: 3	 iter: 3000	 loss:0.25715
epoch: 3	 iter: 4000	 loss:0.26471
epoch: 3	 iter: 5000	 loss:0.26122
epoch: 3	 iter: 6000	 loss:0.26604
epoch: 4	 iter: 1000	 loss:0.15247
epoch: 4	 iter: 2000	 loss:0.19329
epoch: 4	 iter: 3000	 loss:0.19426
epoch: 4	 iter: 4000	 loss:0.19091
epoch: 4	 iter: 5000	 loss:0.19062
epoch: 4	 iter: 6000	 loss:0.18254
Finished Training
Accuracy of plane: 	 87.6%
Accuracy of car: 	 94.0%
Accuracy of bird: 	 83.2%
Accuracy of cat: 	 83.0%
Accuracy of deer: 	 78.7%
Accuracy of d